# Academic Journal Summarization Modeling - T5 Model Baseline

## Setup

In [ ]:
!pip install -q sentencepiece

In [ ]:

!pip install -q transformers

In [ ]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 1

In [ ]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
# Standard libraries
import os
import pandas as pd

# Datset
from datasets import Dataset

# For counting and checkpoints
import pytz
from datetime import datetime

# Modeling
import tensorflow as tf
import nltk

from sklearn.model_selection import train_test_split

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import evaluate

In [ ]:
# Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Data

In [ ]:
# Woojae's path
# df = pd.read_csv('/content/drive/MyDrive/DATASCI266/266 Project/envisci_articles.csv')
# df = pd.read_csv('/content/drive/MyDrive/266_Project/envisci_articles.csv')

# Brendan's path
df = pd.read_csv('/content/drive/MyDrive/DATASCI266/envisci_articles.csv')

# Summer's path
#df = pd.read_csv('/content/drive/MyDrive/266_Project/envisci_articles.csv')

In [ ]:
# Split data
train, test = train_test_split(df, test_size=0.2, random_state=1)

## Baseline

In [ ]:
# Index range
beginning_range_index = 0
ending_range_index = 419

# Output paths - Baseline
LOCAL_OUTPUT_CSV_PATH = './drive/MyDrive/DATASCI266/model_baseline_t5_files/'

In [ ]:
# Initialize the T5 tokenizer and model

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# model_location = './drive/MyDrive/DATASCI266/model_finetuned_t5_models/t5_finetune_no_prompt'

# model = AutoModelForSeq2SeqLM.from_pretrained(model_location)
# tokenizer = AutoTokenizer.from_pretrained(model_location)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
def tokenize_text(text):
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors="pt")
    return inputs

# summarize_phrase = 'Summarize:'
summarize_phrase = 'Summarize this scholarly article:'
# summarize_phrase = 'Summarize this scholarly article for someone that wants to learn about environmental science:'

ref_text=[]
for reference in test['abstract']:
    ref_text.append(reference)

lst_untokenized_texts = [summarize_phrase + ' ' + text for text in test['full_text']]
lst_texts_all = [tokenize_text(text) for text in lst_untokenized_texts]
lst_texts = lst_texts_all[beginning_range_index:ending_range_index]

lst_references = ref_text[beginning_range_index:ending_range_index]

# Generating summaries
count = 1

df_final = pd.DataFrame(columns=['GeneratedText', 'ReferenceText'])
max_len = int(512)
min_len = int(max_len/2)

for i, (text, reference) in enumerate(zip(lst_texts, lst_references)):
  # Constants
  data = []
  est = pytz.timezone('US/Eastern')

  # Calculate Summarization Params

  summaries = model.generate(text['input_ids']
                              , max_new_tokens = max_len
                              , min_length = min_len
                              , no_repeat_ngram_size = 4
  )

  output = tokenizer.batch_decode(summaries, skip_special_tokens=True)

  # Write to CSV
  data.append({'GeneratedText':output, 'ReferenceText':reference})
  df = pd.DataFrame(data)
  df_final = pd.concat([df_final, df], ignore_index=True)
  print(f'Finished with Appending Record: {count}')
  count = count + 1

# Create Filename
file_name = f't5_baseline_{beginning_range_index}_to_{ending_range_index}_summarizephrase_{summarize_phrase}_maxlen{max_len}.csv'
output_path = LOCAL_OUTPUT_CSV_PATH + file_name

df_final.to_csv(output_path, index=False)

Finished with Appending Record: 1
Finished with Appending Record: 2
Finished with Appending Record: 3
Finished with Appending Record: 4
Finished with Appending Record: 5
Finished with Appending Record: 6
Finished with Appending Record: 7
Finished with Appending Record: 8
Finished with Appending Record: 9
Finished with Appending Record: 10
Finished with Appending Record: 11
Finished with Appending Record: 12
Finished with Appending Record: 13
Finished with Appending Record: 14
Finished with Appending Record: 15
Finished with Appending Record: 16
Finished with Appending Record: 17
Finished with Appending Record: 18
Finished with Appending Record: 19
Finished with Appending Record: 20
Finished with Appending Record: 21
Finished with Appending Record: 22
Finished with Appending Record: 23
Finished with Appending Record: 24
Finished with Appending Record: 25
Finished with Appending Record: 26
Finished with Appending Record: 27
Finished with Appending Record: 28
Finished with Appending Recor